In [25]:
%load_ext autoreload
%autoreload 2
import pickle
import glob
import sys
import numpy as np
import pyccl as ccl
sys.path.append('../modules/')
import ModelClusterAbundance
import simulation
import matplotlib.pyplot as plt
import _run_simulations
def load(filename, **kwargs):
    with open(filename, 'rb') as fin:
        return pickle.load(fin, **kwargs)
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define Count and lensing mass models

In [6]:
ClusterAbundanceObject = ModelClusterAbundance.UniversePrediction( default_config_path='../config/capish.ini')

In [7]:
params_default = ClusterAbundanceObject.params_default 
cosmo_fid = params_default['CCL_cosmology']
params_new = params_default
compute_new= {'compute_dNdzdlogMdOmega':False,'compute_richness_mass_relation':False, 
               'compute_completeness':False, 'compute_purity':False ,'compute_halo_bias':False,
             'compute_dNdzdlogMdOmega_log_slope': False}
adds_new={'add_purity':True, 'add_completeness':True}

In [8]:
adds_new={'add_purity':True, 'add_completeness':True}
N1 = ClusterAbundanceObject.model_count(params_new, compute_new, adds_new)
adds_new={'add_purity':True, 'add_completeness':False}
N2 = ClusterAbundanceObject.model_count(params_new, compute_new, adds_new)
adds_new={'add_purity':False, 'add_completeness':True}
N3 = ClusterAbundanceObject.model_count(params_new, compute_new, adds_new)
adds_new={'add_purity':False, 'add_completeness':False}
N4 = ClusterAbundanceObject.model_count(params_new, compute_new, adds_new)

In [ ]:
res_no_completeness = load('/pbs/throng/lsst/users/cpayerne/capish/validation_standard/sims_no_completeness.pkl')
res_no_purity = load('/pbs/throng/lsst/users/cpayerne/capish/validation_standard/sims_no_purity.pkl')
res_incomplete_impure = load('/pbs/throng/lsst/users/cpayerne/capish/validation_standard/sims_incomplete_impure.pkl')
res_no_completeness_no_purity = load('/pbs/throng/lsst/users/cpayerne/capish/validation_standard/sims_no_completeness_no_purity.pkl')

In [ ]:
N_no_completeness_no_purity = np.average(np.array(res_no_completeness_no_purity)[:,0], axis=0)
N_incomplete_impure = np.average(np.array(res_incomplete_impure)[:,0], axis=0)
N_no_purity = np.average(np.array(res_no_purity)[:,0], axis=0)
N_no_completeness = np.average(np.array(res_no_completeness)[:,0], axis=0)

In [ ]:
N_no_completeness_no_purity

In [ ]:
plt.figure(figsize=(6,4))
from matplotlib.ticker import ScalarFormatter
plt.gca().xaxis.set_major_formatter(ScalarFormatter())
plt.gca().xaxis.set_minor_formatter(ScalarFormatter())
plt.ticklabel_format(style='plain', axis='x')
for i in range(3,4):
    alls = N4[:,i]
    plt.plot(ClusterAbundanceObject.richness_centers,N2[:,i]/alls, 'C0', label = 'TH, complete, not pure')
    plt.plot(ClusterAbundanceObject.richness_centers,N3[:,i]/alls, 'C1',label = 'TH, pure, not complete')
    plt.plot(ClusterAbundanceObject.richness_centers,N1[:,i]/alls, 'C2',label = 'TH, not complete, not pure', alpha=0.6, color='grey')
    
    plt.plot(ClusterAbundanceObject.richness_centers,N_no_completeness[:,i]/alls, '--C0', lw=2, label = '30xCAPISH complete, not pure')
    plt.plot(ClusterAbundanceObject.richness_centers,N_no_purity[:,i]/alls, '--C1',lw=2, label = '30xCAPISH pure, not complete')
    plt.plot(ClusterAbundanceObject.richness_centers,N_incomplete_impure[:,i]/alls, '--C2', lw=2,label = '30xCAPISH not complete, not pure')

    
    plt.plot(ClusterAbundanceObject.richness_centers,N_no_completeness_no_purity[:,i]/alls, '--k', label = '30xCAPISH pure, complete')
    break
plt.xlabel('richness')
plt.ylabel('X/(TH, complete+pure)', fontsize=15)
plt.legend()
plt.tight_layout()
#plt.xscale('log')
plt.xlim(20, 180)